# Add/Remove Machine to Device Groups based on Machine Tag

    Author: Jing Nghik
    Last Updated 6/23/2020

Purpose: This notebook is used to add device's to a certain group based on the machine tag.

## Requirements: 

[Machine groups already pre-configured by Device tag](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/machine-groups)

    You need to have the machine groups already configured that filter based on the device tag element.

[Proper MDATP API Token](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/exposed-apis-create-app-webapp)
        
    Microsoft Defender ATP exposes much of its data and actions through a set of programmatic APIs. Those APIs will help you automate work flows and innovate based on Microsoft Defender ATP capabilities.

## Import required modules and credentials 

In [4]:
$VerbosePreference = "ignore"   ## Set to 'continue to get verbose output'

## This will run a script to import the required modules and credentials stored in the "credential" file
$scriptPath = "~/Notebooks/PowerShell/custom-modules/import.ps1"; . $scriptPath  # this will run the script


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Function        Get-DecryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-EncryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-HashOfString                                   0.0        pp-core
Function        Get-MasterPassword                                 0.0        pp-core
Function        Get-StoredCredentials                              0.0        pp-core
Function        Invoke-CheckCredentials                            0.0        pp-core
Function        New-StoreCredentials                               0.0        pp-core
Function        Get-MSGraphAuthToken                               0.0        graph
Function        Invoke-GraphAuthentication                         0.0        graph
Function        Invoke-MSGraphQuery                        

## Authenticate with Graph API and get access Token for MDATP Resource

In [5]:
## Get a token if authenticates properly. 

$authParams = @{
    tenantId = $credentials.tenantId
    client_id = $credentials.client_id
    client_secret = $credentials.client_secret
    resource = "https://api.securitycenter.microsoft.com"    ## resource Dont change since we want to query MDATP REST API Resource
    grant_type = "client_credentials"  ## This is using a device_code flow (supports: client_credentials, device_code)
    #scope = "https://api.securitycenter.microsoft.com/Machine.ReadWrite.All" ## set the scope
};

## this will authenticate with Graph API and token will be stored in <tokenResponse> variable.
$token = (Invoke-GraphAuthentication -authParams $authParams).access_token
$token


Authentication Parameters detected
Personal Access Token (PAT) grant_type

----------------------------------------------------------------------------
Authentiating with Microsoft Graph API using a Personal Access Token (PAT)
https://docs.microsoft.com/en-us/azure/storage/common/storage-auth-aad-app
----------------------------------------------------------------------------
Requesting Token at https://login.microsoftonline.com/e4194ef7-b5d7-4db1-b7e9-a73af406f276/oauth2/token


Received Token!
Connected and Access Token received and will expire 1612917847
eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Im5PbzNaRHJPRFhFSzFqS1doWHNsSFJfS1hFZyIsImtpZCI6Im5PbzNaRHJPRFhFSzFqS1doWHNsSFJfS1hFZyJ9.eyJhdWQiOiJodHRwczovL2FwaS5zZWN1cml0eWNlbnRlci5taWNyb3NvZnQuY29tIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvZTQxOTRlZjctYjVkNy00ZGIxLWI3ZTktYTczYWY0MDZmMjc2LyIsImlhdCI6MTYxMjkxMzk0NywibmJmIjoxNjEyOTEzOTQ3LCJleHAiOjE2MTI5MTc4NDcsImFpbyI6IkUyWmdZUGltbGYwc3FmbGE4bytiRm8rZmhZZmVCUUE9IiwiYXBwaWQiOiJjZjg5Yz

## Authenticate with Graph API and get access Token for MDATP Resource

In [6]:
## This is the authentication header for REST API Call
$authHeader = @{ 
    'Content-Type' = 'application/json'
    Accept = 'application/json'
    Authorization = "Bearer $token" 
}

## The following commands will query the graph API and output the first 5 machines in table format
$URI = "https://api.securitycenter.windows.com/api/machines"
$Response = (Invoke-RestMethod -Method GET -Uri $URI -Headers $authHeader -body $body -ErrorAction Stop)
$Response.value | Select-Object -First 5 | Select-Object @('id','computerDnsName','machineTags') | FT


id                                       computerDnsName machineTags
--                                       --------------- -----------
cb5402466f9427f3ea03c81c872ff9be0ce4b03b w10-jingtoso    {group1}



## Identify machine, tag, and whether to add or remove the tag on the machine

In [7]:
$machineId = "cb5402466f9427f3ea03c81c872ff9be0ce4b03b"  ## Machine ID to add or remove device group
$machineTag = "boob"
$action = "Add" ## This can be add or remove

## Add/Remove machine based on above variables

In [8]:
####### Creates the json body ####################
$jsonBody = @{
    Value = $machineTag
    Action = $action
}
$jsonBody = ($jsonbody | ConvertTo-JSON); $jsonBody

####### Calls MDATP Rest API and either add or remove machine tag #################
$URI = "https://api-us.securitycenter.windows.com/api/machines/$machineId/tags"; $URI
$Response = Invoke-RestMethod -Method POST -Uri $URI -Headers $authHeader -body $jsonBody -ErrorAction Stop

{
  "Action": "Add",
  "Value": "boob"
}
https://api-us.securitycenter.windows.com/api/machines/cb5402466f9427f3ea03c81c872ff9be0ce4b03b/tags


## Get Machine list again to verify

In [9]:
$URI = "https://api-us.securitycenter.windows.com/api/machines"
$Response = (Invoke-RestMethod -Method GET -Uri $URI -Headers $authHeader -body $body -ErrorAction Stop)
$Response.value | Select-Object @('id','computerDnsName','machineTags') | FT


id                                       computerDnsName machineTags
--                                       --------------- -----------
cb5402466f9427f3ea03c81c872ff9be0ce4b03b w10-jingtoso    {boob, group1}

